<a href="https://colab.research.google.com/github/SheidaTalei/FinalProject/blob/main/SheidaTalei_XGboost_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUBJECT: XGBoost
### AUTHOR: Sheida Talei

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#Convert a collection of raw documents to a matrix of TF-IDF features.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import regex
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import re
import string
from xgboost import XGBClassifier

In [ ]:
#------------------------------------------Empty Rows Removal----------------------------------------------------------------
# This function Will: 1- Remove all empty rows from csv file 2- Save data to the same csv
def removeEmptyRows(fileName):
    df = pd.read_csv(fileName, encoding='utf-8-sig')
    df = df.dropna(subset=['text'], how='all', axis=0) 
    df.to_csv(fileName, header=True, encoding='utf-8-sig',  index=False)
    
    return df

In [ ]:
# df1= removeEmptyRows ('/content/drive/MyDrive/Final/Prepared_Data_1_Labeled.csv')
# df2 =  removeEmptyRows ('/content/drive/MyDrive/Final/Prepared_train_Data_test#1.csv')

In [ ]:
# X_and_Y =pd.concat([df1, df2], ignore_index=True, names=['tweeter_handle', 'text', 'Label', 'statues_count' ])
X_and_Y = removeEmptyRows ('/content/drive/MyDrive/Final/train_temp.csv')

Y= X_and_Y.Label
X = X_and_Y.text

In [ ]:
# X_and_Y_test = removeEmptyRows('/content/drive/MyDrive/Final/Prepared_test_Data_test#1.csv')
X_and_Y_test = removeEmptyRows('/content/drive/MyDrive/Final/test_temp.csv')
Y_test= X_and_Y_test.Label
X_test = X_and_Y_test.text

In [ ]:
#--------------------------------------------------Loading StopWords ------------------------------------------
#Source of file: https://sites.google.com/site/kevinbouge/stopwords-lists
def getStopWord ():
    try:
        file = open('/content/drive/MyDrive/Final/stopwords_fa.txt', 'r', encoding='utf-8-sig')
        file_readed = file.read()
    
    finally:
        file.close()
        
    stopWord_Set = set(file_readed.split())
    return stopWord_Set


In [ ]:
persian_stop_word = list(getStopWord())

In [ ]:
count_0 = 0
count_1 = 0

for i in Y:
    if (i==0):
        count_0 = count_0+1
    else:
        count_1 = count_1 + 1
        

In [ ]:
print(count_0)
print(count_1)

23658
24624


In [9]:
# import seaborn as sns

# sns.countplot(x='Label', data=X_and_Y)

In [ ]:
#source: https://xgboost.readthedocs.io/en/latest/parameter.html


XGB_classifier = XGBClassifier(scale_pos_weight = count_1/count_0, n_estimators=500 , max_depth=8 , subsample=0.8 , eta = 0.1, verbosity=1 )

# scale_pos_weight = 2237/7713

#Source: https://stackoverflow.com/questions/44066264/how-to-choose-parameters-in-tfidfvectorizer-in-sklearn-during-unsupervised-clust



vectorizer =  TfidfVectorizer(stop_words=persian_stop_word ,  max_features=2000 )
X_train  = vectorizer.fit_transform(X)

# pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer(stop_words=persian_stop_word )),
#     ('clf',XGB_classifier),
# ])
# parameters = {
#     'tfidf__max_df': (0.25),
#     'tfidf__max_features': (100)
   
#     # 'clf__max_depth' :(3,5,8), 
#     # 'clf__reg_alpha': (1.0,0.5,0.1), 
#     # 'clf__subsample':(0.1,0.5,0.8), 
#     # 'clf__eta':(0.01,0.05,0.1)
# }

# grid_search_tune = GridSearchCV(pipeline, parameters, cv=20)
# XGBoost = grid_search_tune.fit(X, Y)

XGBoost = XGB_classifier.fit (X_train , Y )

# print (grid_search_tune.best_estimator_.steps)



In [ ]:

X_test_vectorize  = vectorizer.fit_transform(X_test)

In [ ]:
y_predict_XGBoost= XGBoost.predict(X_test_vectorize)


print ('Accuracy for XGBoost: ',accuracy_score(Y_test, y_predict_XGBoost))
print('F1-score for XGBoost: ', f1_score(Y_test, y_predict_XGBoost))
print('roc_auc_score for XGBoost: ', roc_auc_score(Y_test, y_predict_XGBoost))

Accuracy for XGBoost:  0.5332886933231299
F1-score for XGBoost:  0.582955808188734
roc_auc_score for XGBoost:  0.5326877324236479
